In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import numpy as np

In [18]:
# Define image dimensions and parameters
img_height = 224
img_width = 224
batch_size = 32
num_classes = 4

# Define path to your dataset
data_dir = 'Image_Data'

# Prepare the ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1./255)

# Load all data and labels
all_data = []
all_labels = []

categories = ["Blue_fire","Green_fire","Orange_fire","White_fire"]

for category in categories:
    category_path = os.path.join(data_dir, category)
    
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        
        all_data.append(img_array)
        all_labels.append(category)


# Convert lists to numpy arrays
all_data = np.array(all_data)
#all_labels = np.array(all_labels)

In [19]:
# Convert labels to one-hot encoding
label_map = {name: index for index, name in enumerate(categories)}
all_labels = np.array([label_map[label] for label in all_labels])
all_labels = tf.keras.utils.to_categorical(all_labels, num_classes)

In [20]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_data, all_labels, test_size=0.2, random_state=42)

# # Convert labels to one-hot encoding
# y_train = tf.keras.utils.to_categorical(y_train, num_classes)
# y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Preprocess the testing set
test_datagen = ImageDataGenerator()

In [21]:
# Load and preprocess training and testing data
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

# Load the MobileNetV2 model, excluding the top layers
#base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
#                                               include_top=False,
 #                                              weights='imagenet')

base_model = tf.keras.applications.VGG16(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')


In [22]:
# Freeze the base model
base_model.trainable = False

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=test_generator,
    validation_steps=len(X_test) // batch_size,
    epochs=15
)

Epoch 1/15
10/10 [==============================] - 167s 17s/step - loss: 4.6931 - accuracy: 0.4503 - precision_3: 0.4497 - recall_3: 0.4437 - val_loss: 2.3870 - val_accuracy: 0.5938 - val_precision_3: 0.5968 - val_recall_3: 0.5781
Epoch 2/15
10/10 [==============================] - 172s 18s/step - loss: 2.1911 - accuracy: 0.5960 - precision_3: 0.6138 - recall_3: 0.5894 - val_loss: 0.6133 - val_accuracy: 0.7656 - val_precision_3: 0.8305 - val_recall_3: 0.7656
Epoch 3/15
10/10 [==============================] - 222s 23s/step - loss: 1.1625 - accuracy: 0.7156 - precision_3: 0.7252 - recall_3: 0.7094 - val_loss: 0.4197 - val_accuracy: 0.9219 - val_precision_3: 0.9153 - val_recall_3: 0.8438
Epoch 4/15
 3/10 [========>.....................] - ETA: 2:17 - loss: 0.7698 - accuracy: 0.7917 - precision_3: 0.7935 - recall_3: 0.7604

In [ ]:
# Save the model
model.save('VGG16_image_classifier.h5')

In [ ]:
# Evaluate the model on the testing data
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')
print(f'Test precision: {test_precision}')
print(f'Test recall: {test_recall}')

3/3 [==============================] - 11s 4s/step - loss: 0.2791 - accuracy: 0.9048 - precision_2: 0.9268 - recall_2: 0.9048
Test loss: 0.2791098952293396
Test accuracy: 0.9047619104385376
Test precision: 0.9268292784690857
Test recall: 0.9047619104385376
